In [1]:
from google.colab import files
uploaded = files.upload()


Saving vgsales.csv to vgsales.csv


In [9]:
import pandas as pd
# Leer archivo CSV
df = pd.read_csv('vgsales.csv')

# Mostrar las primeras filas del dataset para asegura de que se cargó correctamente
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
pip install pandas scikit-learn

In [10]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Cargar el dataset
data = pd.read_csv('vgsales.csv')

# ------------------------------
# LABEL ENCODING
# ------------------------------
# Instanciar el LabelEncoder
label_encoder = LabelEncoder()

# Aplicar LabelEncoder a 'Genre'
data['Genre'] = label_encoder.fit_transform(data['Genre'])

# Mostrar las clases codificadas para 'Genre' y sus correspondientes valores
genre_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

print("Asignación de valores a cada género:")
for genre, value in genre_mapping.items():
    print(f"{genre}: {value}")

print(data[['Name', 'Genre']].head())

Asignación de valores a cada género:
Action: 0
Adventure: 1
Fighting: 2
Misc: 3
Platform: 4
Puzzle: 5
Racing: 6
Role-Playing: 7
Shooter: 8
Simulation: 9
Sports: 10
Strategy: 11
                       Name  Genre
0                Wii Sports     10
1         Super Mario Bros.      4
2            Mario Kart Wii      6
3         Wii Sports Resort     10
4  Pokemon Red/Pokemon Blue      7


In [11]:
# ------------------------------
# ONE HOT ENCODING
# ------------------------------
# Instanciar el OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# Aplicar OneHotEncoder a 'Platform'
platform_encoded = onehot_encoder.fit_transform(data[['Platform']])

# Convertir el resultado a DataFrame
platform_df = pd.DataFrame(platform_encoded, columns=onehot_encoder.get_feature_names_out(['Platform']))

# Concatenar el DataFrame original con las nuevas columnas codificadas
data_encoded = pd.concat([data, platform_df], axis=1)
data_encoded.drop('Platform', axis=1, inplace=True)  # Eliminar la columna original

# Mostrar los primeros registros del DataFrame con One Hot Encoding
print(data_encoded.head())

   Rank                      Name    Year  Genre Publisher  NA_Sales  \
0     1                Wii Sports  2006.0     10  Nintendo     41.49   
1     2         Super Mario Bros.  1985.0      4  Nintendo     29.08   
2     3            Mario Kart Wii  2008.0      6  Nintendo     15.85   
3     4         Wii Sports Resort  2009.0     10  Nintendo     15.75   
4     5  Pokemon Red/Pokemon Blue  1996.0      7  Nintendo     11.27   

   EU_Sales  JP_Sales  Other_Sales  Global_Sales  ...  Platform_SAT  \
0     29.02      3.77         8.46         82.74  ...           0.0   
1      3.58      6.81         0.77         40.24  ...           0.0   
2     12.88      3.79         3.31         35.82  ...           0.0   
3     11.01      3.28         2.96         33.00  ...           0.0   
4      8.89     10.22         1.00         31.37  ...           0.0   

   Platform_SCD  Platform_SNES  Platform_TG16  Platform_WS  Platform_Wii  \
0           0.0            0.0            0.0          0.0      

In [12]:
# ------------------------------
# NORMALIZACION
# ------------------------------
# Convertir espacios vacíos a NaN
data_encoded.replace("", np.nan, inplace=True)

# Imputar valores faltantes en las columnas de ventas
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']] = imputer.fit_transform(
    data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
)

# Normalizar las columnas de ventas
scaler = MinMaxScaler()
data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']] = scaler.fit_transform(
    data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
)

# Redondear a 2 decimales
data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']] = data_encoded[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']].round(2)

# Mostrar los primeros registros
print(data_encoded.head())

   Rank                      Name    Year  Genre Publisher  NA_Sales  \
0     1                Wii Sports  2006.0     10  Nintendo      1.00   
1     2         Super Mario Bros.  1985.0      4  Nintendo      0.70   
2     3            Mario Kart Wii  2008.0      6  Nintendo      0.38   
3     4         Wii Sports Resort  2009.0     10  Nintendo      0.38   
4     5  Pokemon Red/Pokemon Blue  1996.0      7  Nintendo      0.27   

   EU_Sales  JP_Sales  Other_Sales  Global_Sales  ...  Platform_SAT  \
0      1.00      0.37         0.80          1.00  ...           0.0   
1      0.12      0.67         0.07          0.49  ...           0.0   
2      0.44      0.37         0.31          0.43  ...           0.0   
3      0.38      0.32         0.28          0.40  ...           0.0   
4      0.31      1.00         0.09          0.38  ...           0.0   

   Platform_SCD  Platform_SNES  Platform_TG16  Platform_WS  Platform_Wii  \
0           0.0            0.0            0.0          0.0      

In [13]:
# ------------------------------
# GUARDAR EL RESULTADO EN UN NUEVO CSV
# ------------------------------
# Guardar el DataFrame procesado en un nuevo archivo CSV
data_encoded.to_csv('vgsales_encoded.csv', index=False)



In [14]:
# ------------------------------
# DISCRETIZACION
# ------------------------------
import pandas as pd

# Cargar el dataset
data = pd.read_csv('vgsales.csv')

# Define los límites de los bins y las etiquetas correspondientes
bins = [0, 10, 25, 50, 75, 100]  # Ajuste de los límites
labels = ['Bajas Ventas', 'Moderadas Ventas', 'Altas Ventas', 'Muy Altas Ventas', 'Epicas Ventas']  # 5 etiquetas para 6 bins

# Discretización de la columna Global_Sales
data['Global_Sales_Disc'] = pd.cut(data['Global_Sales'], bins=bins, labels=labels, include_lowest=True)

# Mostrar los primeros registros del DataFrame
print(data[['Name', 'Global_Sales', 'Global_Sales_Disc']].head(10))  # Muestra más registros para ver la clasificación

# Guardar el DataFrame modificado en un nuevo archivo CSV
data.to_csv('vgsales_discretized.csv', index=False)


                        Name  Global_Sales Global_Sales_Disc
0                 Wii Sports         82.74     Epicas Ventas
1          Super Mario Bros.         40.24      Altas Ventas
2             Mario Kart Wii         35.82      Altas Ventas
3          Wii Sports Resort         33.00      Altas Ventas
4   Pokemon Red/Pokemon Blue         31.37      Altas Ventas
5                     Tetris         30.26      Altas Ventas
6      New Super Mario Bros.         30.01      Altas Ventas
7                   Wii Play         29.02      Altas Ventas
8  New Super Mario Bros. Wii         28.62      Altas Ventas
9                  Duck Hunt         28.31      Altas Ventas
